## 🙄 4차 산업혁명에 관심이 많아 이에 대해 본격적인 분석에 앞서 네이버 검색 API를 통해 간략적인 대중들의 생각을 파악해보고자 한다.





In [1]:
import pandas as pd
import json
import math
import re
import csv
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

In [2]:
naver_client_id = "dABwTg5q0WrSjsUsWsDU"
naver_client_secret = "62yEiEGvU4"

### 🏹 get_blog_count 함수는 블로그 내에서 원하는 query별로 탐색하는 과정으로 블로그 개수를 나타내는 로직을 담았다.

In [3]:
def get_blog_count(query, display):
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code == 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        print("Last build date: " + str(response_body_dict['lastBuildDate']))
        print("Total: " + str(response_body_dict['total']))
        print("Start: " + str(response_body_dict['start']))
        print("Display: " + str(response_body_dict['display']))

        if response_body_dict['total'] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict['total'] / int(display))

            if blog_total >= 1000:
                blog_count = 1000
            else:
                blog_count = blog_total

            print("Blog total: " + str(blog_total))
            print("Blog count: " + str(blog_count))

        return blog_count

## 🏹 get_blog_post 함수는 각 블로그마다 원하는 쿼리단어가 포함되었으면 이를 csv로 추출하는 로직을 담았다.

In [4]:
def get_blog_post(query, display, start_index, sort):
    global no, df
    
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort

    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code == 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))
        for item_index in range(0, len(response_body_dict['items'])):
            try:
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['title'])
                link = response_body_dict['items'][item_index]['link'].replace("amp;", "")
                description = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']

                no += 1
                post_code = requests.get(link)
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text, 'lxml')

                blog_post_content_text = ""
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com" + mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text, 'lxml')
                    
                    for blog_post_content in blog_post_soup.find_all('div', class_='se-viewer'):
                        blog_post_content_text = blog_post_content.get_text()
                        
                    for blog_post_content in blog_post_soup.find_all('div', class_='se_doc_viewer'):
                        blog_post_content_text = blog_post_content.get_text()
 
                    for blog_post_content in blog_post_soup.select('div#postViewArea'):
                        blog_post_content_text = blog_post_content.get_text()

                df.loc[no] = [title, link, description, blogger_name, blogger_link, post_date, blog_post_content_text]
                print("*", end='')
                
            except:
                item_index += 1

In [5]:
no = 0                 # 몇개의 포스트를 저장하였는지 세기 위한 index
query = "4차 산업혁명"   # 검색을 원하는 문자열로서 UTF-8로 인코딩한다.
display = 10           # 검색 결과 출력 건수 지정, 10(기본값),100(최대)
start = 1              # 검색 시작 위치로 최대 1000까지 가능
sort = "sim"           # 정렬 옵션: sim(유사도순, 기본값), date(날짜순)

# 블로그를 DataFrame에 저장
df = pd.DataFrame(columns=("Title", "Link", "Description", "Blogger Name", "Blogger Link", "Post Date", "Post Contents"))

blog_count = get_blog_count(query, display)
for start_index in range(start, blog_count + 1, display):
    get_blog_post(query, display, start_index, sort)

Last build date: Sat, 22 Jan 2022 12:04:42 +0900
Total: 746998
Start: 1
Display: 10
Blog total: 74700
Blog count: 1000
*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [6]:
df.to_csv("data/fourth_revolution.csv", header=True, index=False, encoding="utf-8")

In [7]:
#상위 인덱스 5개 추출
result_data = pd.read_csv("data/fourth_revolution.csv", encoding="utf-8")
result_data[0:5]

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,4차산업혁명(4IR)과 의료기술 과목,https://blog.naver.com/ymkim1959?Redirect=Log&...,4차산업혁명의 정의는 무엇인가요? 이를 구현하기 위한 기술은 무엇이 필요한가요? 기...,4차산업혁명 시대의 의공학,https://blog.naver.com/ymkim1959,20211221,\n\n\n\n\n\n\n4차산업혁명\n\n\n\n\n4차산업혁명(4IR)과 의료기...
1,2030년 제4차산업혁명,https://blog.naver.com/hott0116?Redirect=Log&l...,시기 4차 산업혁명은 정보통신의 융합이 이루어지는 시기 그동안 우리의 생활과 산업은...,초긍정녀의 다이어리,https://blog.naver.com/hott0116,20211004,\n\n\n\n\n\n\n일상과 생각♭\n\n\n\n\n2030년 제4차산업혁명 ...
2,"4차 산업혁명 시대, 드론을 더 안전하게 활용하는 방법은?",https://blog.naver.com/kcscpr?Redirect=Log&log...,"드론, 4차 산업혁명의 양날의 검! 무인항공기라 불리는 일명 '드론', 드론은 4차...",LX 한국국토정보공사 블로그,https://blog.naver.com/kcscpr,20211116,"\n\n\n\n\n\n\nLX 랜드러버\n\n\n\n\n4차 산업혁명 시대, 드론을..."
3,영재교육 4차산업혁명에 생겨나는 미래 유망 직업,https://blog.naver.com/cmath_club?Redirect=Log...,"클라우드와 에지 컴퓨팅 등의 기술을 통하여 정보 시스템,통신 시스템,산업용 관리 시...",사고력수학 전문가 시매쓰,https://blog.naver.com/cmath_club,20220117,\n\n\n\n\n\n\n\n\n\n영재교육\n\n\n\n\n영재교육 4차산업혁명에...
4,4차산업혁명 시대에 창의력은 무엇일까?,https://blog.naver.com/simon9627?Redirect=Log&...,"그러나 시대가 완전히 바뀐 4차산업혁명 시대에는 앞서 말했던 미술, 음악, 무용 등...",스마트메이커,https://blog.naver.com/simon9627,20211211,\n\n\n\n\n\n\n이슈 뉴스\n\n\n\n\n4차산업혁명 시대에 창의력은 무...


In [8]:
#요약 내용 상위 5개 인덱스 추출
content = result_data.get('Description')
content[0:5]

0    4차산업혁명의 정의는 무엇인가요? 이를 구현하기 위한 기술은 무엇이 필요한가요? 기...
1    시기 4차 산업혁명은 정보통신의 융합이 이루어지는 시기 그동안 우리의 생활과 산업은...
2    드론, 4차 산업혁명의 양날의 검! 무인항공기라 불리는 일명 '드론', 드론은 4차...
3    클라우드와 에지 컴퓨팅 등의 기술을 통하여 정보 시스템,통신 시스템,산업용 관리 시...
4    그러나 시대가 완전히 바뀐 4차산업혁명 시대에는 앞서 말했던 미술, 음악, 무용 등...
Name: Description, dtype: object